# Step 4: Model Development

---

Appropriate machine learning algorithms for classification problems like ours includes:

- Logistic Regression
- Decision Tree
- Random Forest
- k-Nearest Neighbors
- Support Vector Machine
- Naive Bayes

Considering the small number of samples in our dataset, the number of features, the binary nature of our problem, and the assignment constraints, we will develop and pick out the best algorithms among these 4:

- Logistic Regression
- Decision Tree
- Bagged Trees
- Random Forest

For individual model's performance evaluation, we will be using [F1 score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) to measure our models' performance and optimize them. While every sepsis case needs immediate attention, thus every potential sepsis patient needs close monitering, false positive predictions might take medical resources away from patients that might actually need it. In subsequent model development, if we see that recall and precision have different weights in our problem domain, we can switch to [F-beta](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.fbeta_score.html#:~:text=The%20F-beta%20score%20is,recall%20in%20the%20combined%20score.) instead.

To get this performance, we will use K-Fold cross validation.

For regularization, we will be using Grid Search method to select and fine tune our hyperparameters.

To compare between the different model of different algorithms, we will use both $F_1$ score and [ROC-AUC](https://www.youtube.com/watch?v=4jRBRDbJemM). This will be done after we have developed our models.

---

**Table of Contents**

1. [Data Splitting](#data)
2. [Logistic Regression](#logistic)
3. [Decision Tree](#tree)
4. [Bagged Trees](#bagged-trees)
5. [Random Forest](#forest)

In [ ]:
# Imports and environment setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')


from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier


# Custom utils functions
from utils.visualization import show_train_val_confusion_matrix
from utils.visualization import visualize_training

We will store our developed models and their performance in the following variables:

In [ ]:
clfs = dict()

## 1. Data Splitting<a id="data"></a>

For this section, we are using the processed datasets from [Step 3. Data Processing](Step3.DataProcessing.ipynb).

In [ ]:
df_train = pd.read_csv("../data/processed_train.csv")
df_test = pd.read_csv("../data/processed_test.csv")

In [ ]:
df_test.info()
df_test.describe()

In [ ]:
df_train.info()
df_train.describe()

In [ ]:
# Separate our features (X) and target (y)
df_X = df_train.drop(["Sepsis"], axis=1)
df_y = df_train[["Sepsis"]]

# Splitting our training data into a train set and a validation set
train_X, val_X, train_y, val_y = train_test_split(
    df_X, df_y,
    shuffle=True,
    random_state=0,  # Ensure reproducible results
    test_size=0.2    # 80% Train - 20% Validation
)

print(f"train_X shape: {train_X.shape}")
print(f"train_y shape: {train_y.shape}")
print(f"val_X shape: {val_X.shape}")
print(f"val_y shape: {val_y.shape}")

Setting up K-Fold cross validation for our models:

In [ ]:
cv = KFold(
    n_splits=4,     # Each fold is 20% of df_X
    shuffle=True,
    random_state=0  # Reproducible result
)

## 2. Logistic Regression<a id="logistic"></a>

We would develop and evaluate the following models:

- Logistic Regression with Linear Features;
- Logistic Regression with Polynomial Features of degree 2;
- Logistic Regression with Polynomial Features of degree 3;
- Logistic Regression with Polynomial Features of degree 4.

We are experimenting with both $L_1$ (lasso) and $L_2$ (ridge) regularization to find out which method is more appropriate. And although findings in [Step 1. EDA](Step1.EDA.ipynb) suggest that we stop at Polynomial Features of degree 3, we are still experimenting with degree 4 because we introduced regularization into our Logistic Regression model. However, we will not go any higher than degree 4, since it is too computationally expensive.

In [ ]:
clf = LogisticRegression(
    solver="liblinear",       # Good for our small dataset
    class_weight="balanced",
    max_iter=1_000,
    random_state=0            # Reproducible result
)

### 2.1. Hyperparameters

- Regularization strength $\lambda$ (lambda)
- Regularization type: $L_1$ or $L_2$

In [ ]:
params = {
    "C": 1 / np.logspace(-5, 2, num=100),
    "penalty": ["l1", "l2"]
}

### 2.2. Logistic Regression with Linear Features

In [ ]:
grid_clf = GridSearchCV(clf, params, cv=cv, scoring="f1")
grid_clf.fit(train_X, train_y.values.ravel())

train_pred_y = grid_clf.predict(train_X)
val_pred_y = grid_clf.predict(val_X)
clfs["logistic1"] = grid_clf

In [ ]:
visualize_training(clf, grid_clf, train_X, val_X, train_y, val_y, "Training Evaluation\nLogistic Regression - Linear", "../images/ModelDev_Logistic1_TrainValCompare.png")

In [ ]:
show_train_val_confusion_matrix(train_y, train_pred_y, val_y, val_pred_y, "Model Performance\nLogistic Regression - Linear", "../images/ModelDev_Logistic1_CM.png")

### 2.3. Logistic Regression with 2nd Order Polynomial Features

In [ ]:
# Degree 2 Polynomial Features:
poly = PolynomialFeatures(2)
train_X_poly2 = poly.fit_transform(train_X)
val_X_poly2 = poly.transform(val_X)

# We need to scale the data again after transforming it to polynomial features
scaler = StandardScaler()
train_X_poly2 = scaler.fit_transform(train_X_poly2)
val_X_poly2 = scaler.transform(val_X_poly2)

In [ ]:
grid_clf = GridSearchCV(clf, params, cv=cv, scoring="f1")
grid_clf.fit(train_X_poly2, train_y.values.ravel())

train_pred_y = grid_clf.predict(train_X_poly2)
val_pred_y = grid_clf.predict(val_X_poly2)
clfs["logistic2"] = grid_clf

In [ ]:
visualize_training(clf, grid_clf, train_X, val_X, train_y, val_y, "Training Evaluation\nLogistic Regression - Polynomial 2", "../images/ModelDev_Logistic2_TrainValCompare.png")

In [ ]:
show_train_val_confusion_matrix(train_y, train_pred_y, val_y, val_pred_y, "Model Performance\nLogistic Regression - 2nd Order Polynomial", "../images/ModelDev_Logistic2_CM.png")

### 2.4. Logistic Regression with 3rd Order Polynomial Features

In [ ]:
# Degree 3 Polynomial Features:
poly = PolynomialFeatures(3)
train_X_poly3 = poly.fit_transform(train_X)
val_X_poly3 = poly.transform(val_X)

# We need to scale the data again after transforming it to polynomial features
scaler = StandardScaler()
train_X_poly3 = scaler.fit_transform(train_X_poly3)
val_X_poly3 = scaler.transform(val_X_poly3)

In [ ]:
grid_clf = GridSearchCV(clf, params, cv=cv, scoring="f1")
grid_clf.fit(train_X_poly3, train_y.values.ravel())

train_pred_y = grid_clf.predict(train_X_poly3)
val_pred_y = grid_clf.predict(val_X_poly3)
clfs["logistic3"] = grid_clf

In [ ]:
visualize_training(clf, grid_clf, train_X, val_X, train_y, val_y, "Training Evaluation\nLogistic Regression - Polynomial 3", "../images/ModelDev_Logistic3_TrainValCompare.png")

In [ ]:
show_train_val_confusion_matrix(train_y, train_pred_y, val_y, val_pred_y, "Model Performance\nLogistic Regression - 3rd Order Polynomial", "../images/ModelDev_Logistic3_CM.png")

### 2.5. Logistic Regression with 4th Order Polynomial Features

In [ ]:
# Degree 4 Polynomial Features:
poly = PolynomialFeatures(4)
train_X_poly4 = poly.fit_transform(train_X)
val_X_poly4 = poly.transform(val_X)

# We need to scale the data again after transforming it to polynomial features
scaler = StandardScaler()
train_X_poly4 = scaler.fit_transform(train_X_poly4)
val_X_poly4 = scaler.transform(val_X_poly4)

In [ ]:
grid_clf = GridSearchCV(clf, params, cv=cv, scoring="f1")
grid_clf.fit(train_X_poly4, train_y.values.ravel())

train_pred_y = grid_clf.predict(train_X_poly4)
val_pred_y = grid_clf.predict(val_X_poly4)
clfs["logistic4"] = grid_clf

In [ ]:
visualize_training(clf, grid_clf, train_X, val_X, train_y, val_y, "Training Evaluation\nLogistic Regression - Polynomial 4", "../images/ModelDev_Logistic4_TrainValCompare.png")

In [ ]:
show_train_val_confusion_matrix(train_y, train_pred_y, val_y, val_pred_y, "Model Performance\nLogistic Regression - 4th Order Polynomial", "../images/ModelDev_Logistic4_CM.png")

### 2.6. Observation

All models seem to have good fit on the problem, however:

- $L_1$ (Lasso) Regularization clearly is not a good choice for our problem, as the larger $\lambda$ (lambda) grows, the worse our models' performance gets, regardless of which degree of polynomial feature it is. This is likely because we have only a few features, and all of them are significant, while Lasso Regularization tends to feature select.
- Thus $L_2$ (Ridge) Regularization is more appropriate for our problem.
- The best performing model is `clf_logistic4_l2`, in contrary to our hypothesis in [Step 1. EDA](Step1.EDA.ipynb), but this is because we have applied regularization.

## 3. Decision Tree<a id="tree"></a>

### 3.1. Hyperparameters

In [ ]:
params = {
    "max_depth": np.arange(2, 300, 50),
    "min_samples_split": np.arange(2, 50, 5),
}

### 3.2. Model

In [ ]:
clf = DecisionTreeClassifier(
    criterion="gini",         # Good for balanced data (we have already up-sampled out data so it is now balanced)
    class_weight="balanced",
    random_state=0            # Reproducible result
)

grid_clf = GridSearchCV(clf, params, cv=cv, scoring="f1")
grid_clf.fit(train_X, train_y.values.ravel())

train_pred_y = grid_clf.predict(train_X)
val_pred_y = grid_clf.predict(val_X)
clfs["tree"] = grid_clf

In [ ]:
visualize_training(clf, grid_clf, train_X, val_X, train_y, val_y, "Training Evaluation\nDecision Tree", "../images/ModelDev_Tree_TrainValCompare.png")

In [ ]:
show_train_val_confusion_matrix(train_y, train_pred_y, val_y, val_pred_y, "Model Performance\nDecision Tree", "../images/ModelDev_Tree_CM.png")

## 4. Bagged Trees<a id="bagged-trees"></a>

### 4.1. Hyperparameters

In [ ]:
params = {
    "n_estimators": np.arange(5, 25, 2),
    "max_samples": np.arange(0.1, 1.0, 0.1),
}

### 4.2. Model

In [ ]:
clf = BaggingClassifier(
    estimator=clfs["tree"],
    random_state=0  # Reproducible result
)

grid_clf = GridSearchCV(clf, params, cv=cv, scoring="f1")
grid_clf.fit(train_X, train_y.values.ravel())

train_pred_y = grid_clf.predict(train_X)
val_pred_y = grid_clf.predict(val_X)
clfs["bagged_trees"] = grid_clf

In [ ]:
visualize_training(clf, grid_clf, train_X, val_X, train_y, val_y, "Training Evaluation\nBagged Trees", "../images/ModelDev_BaggedTrees_TrainValCompare.png")

In [ ]:
show_train_val_confusion_matrix(train_y, train_pred_y, val_y, val_pred_y, "Model Performance\nBagged Trees", "../images/ModelDev_BaggedTrees_CM.png")

## 5. Random Forest<a id="forest"></a>

### 5.1. Hyperparamerters

In [ ]:
params = {
    "n_estimators": np.arange(1, 500, 20),
    "max_depth": np.arange(2, 300, 50)
}

### 5.2. Model

In [ ]:
clf = RandomForestClassifier(
    criterion="gini",  # Good for balanced data (we have already up-sampled out data so it is now balanced)
    class_weight="balanced",
    random_state=0     # Reproducible result
)

grid_clf = GridSearchCV(clf, params, cv=cv, scoring="f1")
grid_clf.fit(train_X, train_y.values.ravel())

train_pred_y = grid_clf.predict(train_X)
val_pred_y = grid_clf.predict(val_X)
clfs["forest"] = grid_clf

In [ ]:
visualize_training(clf, grid_clf, train_X, val_X, train_y, val_y, "Training Evaluation\nRandom Forest", "../images/ModelDev_RandomForest_TrainValCompare.png")

In [ ]:
show_train_val_confusion_matrix(train_y, train_pred_y, val_y, val_pred_y, "Model Performance\nRandom Forest", "../images/ModelDev_RandomForest_CM.png")